# Backtest results for {NAME}

## Setting up

In [ ]:
# Loads strategy module and other backtest parameters
from tradeexecutor.backtest.report import setup_backtest_report
backtest_setup = setup_backtest_report()

In [ ]:
# Run the backtest
state = backtest_setup.run()

## Results


In [ ]:
### Equity curve

In [ ]:
###

In [ ]:
### Trading performance